In [1]:
from collections import defaultdict, Counter
import pandas as pd
from csv import reader
from heapq import heappush, heappushpop


In [2]:
sample_output = pd.read_csv('SampleOutput.csv')

In [3]:
sample_output

,LocalTime,Symbol,EventType,Side,FillSize,FillPrice,FillExchange,SymbolBought,SymbolSold,SymbolPosition,SymbolNotional,ExchangeBought,ExchangeSold,TotalBought,TotalSold,TotalBoughtNotional,TotalSoldNotional
0,9:30:00.000,AES,TRADE,t,100,11.14,NYSE,0,100,-100,1114.0,0,100,0,100,0.0,1114.0
1,9:30:01.000,AES,TRADE,b,100,11.16,NASDAQ,100,100,0,1116.0,100,0,100,100,1116.0,1114.0
2,9:30:02.000,CX,TRADE,b,100,7.50,NASDAQ,100,0,100,750.0,200,0,200,100,1866.0,1114.0
3,9:30:03.000,CX,TRADE,t,100,7.51,NASDAQ,100,100,0,751.0,200,100,200,200,1866.0,1865.0
4,9:30:04.000,CX,TRADE,b,200,7.50,NASDAQ,300,100,200,1500.0,400,100,400,200,3366.0,1865.0
5,9:30:05.000,CX,TRADE,t,100,7.50,NASDAQ,300,200,100,750.0,400,200,400,300,3366.0,2615.0
6,9:30:06.000,CX,TRADE,t,200,7.48,NASDAQ,300,400,-100,1496.0,400,400,400,500,3366.0,4111.0
7,9:30:07.000,AIG,TRADE,t,100,56.40,NASDAQ,0,100,-100,5640.0,400,500,400,600,3366.0,9751.0
8,9:30:08.000,AIG,TRADE,t,100,56.39,NYSE,0,200,-200,5639.0,0,200,400,700,3366.0,15390.0
9,9:30:09.000,AIG,TRADE,b,100,56.38,NASDAQ,100,200,-100,5638.0,500,500,500,700,9004.0,15390.0


In [4]:
shabi = pd.read_csv('shabi.csv')
shabi

,LocalTime,Symbol,EventType,Side,FillSize,FillPrice,FillExchange,SymbolBought,SymbolSold,SymbolPosition,SymbolNotional,ExchangeBought,ExchangeSold,TotalBought,TotalSold,TotalBoughtNotional,TotalSoldNotional
0,9:30:00.000,AES,TRADE,t,100,11.14,NYSE,0,100,-100,1114.0,0,100,0,100,0.0,1114.0
1,9:30:01.000,AES,TRADE,b,100,11.16,NASDAQ,100,100,0,1116.0,100,0,100,100,1116.0,1114.0
2,9:30:02.000,CX,TRADE,b,100,7.50,NASDAQ,100,0,100,750.0,200,0,200,100,1866.0,1114.0
3,9:30:03.000,CX,TRADE,t,100,7.51,NASDAQ,100,100,0,751.0,200,100,200,200,1866.0,1865.0
4,9:30:04.000,CX,TRADE,b,200,7.50,NASDAQ,300,100,200,1500.0,400,100,400,200,3366.0,1865.0
5,9:30:05.000,CX,TRADE,t,100,7.50,NASDAQ,300,200,100,750.0,400,200,400,300,3366.0,2615.0
6,9:30:06.000,CX,TRADE,t,200,7.48,NASDAQ,300,400,-100,1496.0,400,400,400,500,3366.0,4111.0
7,9:30:07.000,AIG,TRADE,t,100,56.40,NASDAQ,0,100,-100,5640.0,400,500,400,600,3366.0,9751.0
8,9:30:08.000,AIG,TRADE,t,100,56.39,NYSE,0,200,-200,5639.0,0,200,400,700,3366.0,15390.0
9,9:30:09.000,AIG,TRADE,b,100,56.38,NASDAQ,100,200,-100,5638.0,500,500,500,700,9004.0,15390.0


In [422]:
class MedianFillSize:
    def __init__(self):
        self.small = []  
        self.large = []  

    def addFillSize(self, num):
        if len(self.small) == len(self.large):
            heappush(self.large, -heappushpop(self.small, -num))
        else:
            heappush(self.small, -heappushpop(self.large, num))

    def findMedian(self):
        if len(self.small) == len(self.large):
            return int(float(self.large[0] - self.small[0]) / 2.0)
        else:
            return int(float(self.large[0]))

In [423]:
symbol_bought_dic = defaultdict(int)
symbol_sold_dic = defaultdict(int)
ex_bought = defaultdict(int)
ex_sold = defaultdict(int)
total_bought = 0
total_sold = 0
total_bought_notional = 0.0
total_sold_notional = 0.0
fill_size_sum, fill_idx = 0, 0
median_finder = MedianFillSize()

output_handle = open('shabi.csv','w')
with open('SampleInput.csv', 'r') as f:
    csv_reader = reader(f)
    header = next(csv_reader)
    header.extend(['SymbolBought', 'SymbolSold', 'SymbolPosition',
                   'SymbolNotional', 'ExchangeBought', 'ExchangeSold', 'TotalBought',
                   'TotalSold', 'TotalBoughtNotional', 'TotalSoldNotional'])
    output_handle.write(','.join(header)+'\n')
    for row in csv_reader:
        
        row_4 = int(row[4])
        row_5 = float(row[5])
        fill_size_sum += row_4
        median_finder.addFillSize(row_4)
        fill_idx += 1
        exchange = row[-1]
        symbol = row[1]
        

        symbol_notional = row_4 * row_5

        if row[3] == 'b':
            symbol_bought_dic[symbol] += row_4
            ex_bought[exchange] += row_4
            total_bought += row_4
            total_bought_notional += symbol_notional
        else:
            symbol_sold_dic[symbol] += row_4
            ex_sold[exchange] += row_4 
            total_sold += row_4
            total_sold_notional += symbol_notional 
            
        symbol_position = symbol_bought_dic[symbol] - symbol_sold_dic[symbol]
                    
        for _ in [symbol_bought_dic[symbol], symbol_sold_dic[symbol], symbol_position, symbol_notional, ex_bought[exchange], ex_sold[exchange],
                  total_bought, total_sold, total_bought_notional, total_sold_notional]:
            row.append(str(_))

        output_handle.write(','.join(row)+'\n')
        
output_handle.close()

print(f"Processed Trades: {fill_idx}\n")
print(f"Shares Bought: {total_bought}")
print(f"Shares Sold: {total_sold}")
print(f"Total Volume: {total_bought+total_sold}")
print(f"Notional Bought: ${total_bought_notional:.2f}")
print(f"Notional Sold: ${total_sold_notional:.2f}\n")
ex_bought_sorted_key = sorted(dict(ex_bought))
ex_sold_sorted_key = sorted(dict(ex_sold))

print('Per Exchange Volumes:')

for k in ex_bought_sorted_key:
    print(f"{k} Bought: {ex_bought[k]}")
    print(f"{k} Sold: {ex_sold[k]}")

    
print(f"\nAverage Trade Size: {fill_size_sum / fill_idx:.2f}")
print(f"Median Trade Size: {median_finder.findMedian()}")
active_symbol_string = []
combined_dict = Counter(dict(symbol_bought_dic)) + Counter(dict(symbol_sold_dic))
for k in sorted(combined_dict, key=combined_dict.get, reverse=True):
    active_symbol_string.append(f"{k}({combined_dict[k]})")
    
print(f"10 Most Active Symbols: {','.join(active_symbol_string[:10])}")

Processed Trades: 12

Shares Bought: 600
Shares Sold: 800
Total Volume: 1400
Notional Bought: $14642.00
Notional Sold: $21062.00

Per Exchange Volumes:
NASDAQ Bought: 600
NASDAQ Sold: 600
NYSE Bought: 0
NYSE Sold: 200

Average Trade Size: 116.67
Median Trade Size: 100
10 Most Active Symbols: CX(700),AIG(500),AES(200)


In [393]:
trade[trade.isna()['Side'] == True]

,LocalTime,Symbol,EventType,Side,FillSize,FillPrice,FillExchange
301,12:00:00.000,LNCH,TIME,NaN,NaN,NaN,NaN
